In [ ]:
from regain.datasets.hmm import generate_hmm
from regain.hmm.hmm_graphical_lasso import HMM_GraphicalLasso 
import numpy as np
from importlib import reload
import regain.hmm.utils; reload(regain.hmm.utils)
from regain.hmm.utils import results_recap
from regain.hmm.utils import cross_validation#, cross_validation_auto
import pickle as pkl

from sklearn import cluster
from sklearn.mixture import GaussianMixture
#from pomegranate import *

In [ ]:
st = 5
dim =10
mm = 'Normal'
cluster_list = [3,4,5,6,7,8]

res_all = {}



print('Case: N_states:',st,',TS dim:',dim,',Mean mode:',mm)

# Generate a dataset
dataset = generate_hmm(n_samples=2000,
                       n_states=st,
                       n_dim_obs=dim,
                       mode_precisions='regain_random',
                       mode_mean = mm,
                       transition_type='fixed',
                       sigma=1)

In [ ]:
# CV hmm_gmm 

hmm_gmm = HMM_GraphicalLasso(alpha=30,
                             n_clusters=st,
                             verbose=False,
                             mode='scaled',
                             warm_restart=True,
                             repetitions=5,
                             n_jobs=-1)

hmm_gmm.fit(dataset['data'])

print(hmm_gmm.lambdas)

print(results_recap(dataset['states'],hmm_gmm.labels_,dataset['thetas'],hmm_gmm.precisions_))

In [ ]:
hmm_gmm.lambdas

In [ ]:
res = cross_validation(hmm_gmm,
       dataset['data'],
       params={'alpha':np.linspace(15,30 ,10),
               'n_clusters':cluster_list},
       n_repetitions=5)

In [ ]:
with open('Test_bic.pickle', 'wb') as f:
    pkl.dump([dataset,res], f)

In [ ]:
res_alpha_fede = res[1]
couples = []
mean_bic = []
std_bic =[]
dispersion_coeff = []
for k in res_alpha_fede.keys():
    couples.append(k)
    mean_bic.append(res_alpha_fede[k]['mean_bic'])
    std_bic.append(res_alpha_fede[k]['std_bic'])
    dispersion_coeff.append(res_alpha_fede[k]['dispersion_coefficient'])

In [ ]:
cluster_list = [3,4,5,6,7,8]
alpha_list = np.linspace(15,30 ,10)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np


fig = plt.figure()
ax = fig.gca(projection='3d')

# Make data.
X, Y = np.meshgrid(cluster_list, alpha_list)

Z = np.zeros((6, 10)).T
for i, x in enumerate(cluster_list):
    for j, y in enumerate(alpha_list):
        Z[j,i] = res_alpha_fede[(y, x)]['mean_bic']

print(Z)
# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)


# Customize the z axis.
#ax.set_zticklabels([])
ax.set_xlabel('No. states')
ax.set_ylabel(r'$\alpha$')
ax.set_zlabel('BIC', rotation=90)
# ax.zaxis.set_major_locator(LinearLocator(10))
# ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
ax.view_init(25, 70)
# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)
plt.tight_layout()
plt.savefig("/Users/vt908/Dropbox (Personal)/finance_graphical_modles/NeurIPS2020/BIC.pdf", 
            dpi=200, transparent=True,
           bbox_inches='tight')
plt.show()

# plt.figure(figsize=(15,5))
# plt.plot(mean_bic)
# plt.fill_between(np.arange(len(mean_bic)), np.array(mean_bic)-np.array(std_bic),
#                  np.array(mean_bic)+np.array(std_bic), alpha=0.3)
# plt.axvline(np.argmin(mean_bic), color='red')
# plt.xticks(np.arange(len(mean_bic)),couples, rotation='vertical')
# plt.show()

#plt.figure(figsize=(15,10))
#plt.plot(dispersion_coeff)
#plt.axvline(np.argmin(dispersion_coeff), color='red')
#plt.xticks(np.arange(len(mean_bic)),couples, rotation='vertical')
#plt.show()

#plt.figure(figsize=(15,5))
#final_res = np.array(mean_bic/np.sum(mean_bic)) + np.array(dispersion_coeff)
#plt.plot(final_res)
#plt.axvline(np.argmin(final_res), color='red')
#plt.xticks(np.arange(len(mean_bic)),couples, rotation='vertical')
#plt.show()